In [ ]:
#spell_checker
!pip install git+https://github.com/ssut/py-hanspell.git

!pip install konlpy


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
#lib

import re

import pickle

from hanspell import spell_checker
from konlpy.tag import Kkma

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

In [18]:
# word_to_index path
word_to_index_path = '/content/drive/MyDrive/Project/Hel_ri_celus (AI voice_bot for delivery_riders)/Chatbot/data/word_to_index.pkl'

# model_path
model_path = '/content/drive/MyDrive/Project/Hel_ri_celus (AI voice_bot for delivery_riders)/Chatbot/model/best_model.h5'
model = load_model(model_path)

In [19]:
# 정규표현식 함수 정의

def re_sub(self):
    
    sample = re.sub(r',', '', self)
    self= re.sub(r'(.)(\d)\s(\d)\s(\d)\s(\d)', r'\1\2\3\4\5', sample)

    self = re.sub("\d\d\d\d"," @",self)
    self = re.sub("\d\d분"," #분",self)

    return self

In [20]:
## 띄어쓰기, 맞춤법
def check_spell(self):
    
    spelled_sent = spell_checker.check(self)
    self = spelled_sent.checked

    return self

In [21]:
# kkma 형태소 토큰화 함수정의
def kkma_tokenizer(self):
    kkma = Kkma()
    valid_pos = ['NNG','VV','SW','MAG']

    # tokenize
    token_text = kkma.pos(self)
    
    # 불용어 제거
    ls = []
    for token in token_text:
        
        if token[1] in valid_pos:
            ls.append(token[0])
        
    self = ls

    return self

In [22]:
## df의 정수인코딩 column 생성 및 반영 함수화
# tokenizer.fit_on_texts 에서 특수문자 @,# 을 자체적으로 없앰, 이 부분 보완 필요

def int_encode(self):
    
    # word_to_index 불러오기
    wti = open(word_to_index_path, 'rb')
    word_to_index = pickle.load(wti)
    
    # 토크나이저 최적화
    tokenizer = Tokenizer()
    tokenizer.word_index = word_to_index

    seq = tokenizer.texts_to_sequences(self)

    ls = sum(seq,[])

    self = ls

    
    return self

In [23]:
def padding(self):
    max_len = 8

    pad = [0]*(max_len-len(self))
    self = pad + self

    return self

In [24]:
def pred(self):

    self = np.array(self).reshape(-1,8)

    
    pred = model.predict(self)

    
    if np.max(pred[0]) < 0.3:
        self = 99
    else:
        self = np.argmax(pred)

    return self

In [140]:
def start_predict(self):
    
    re_self = re_sub(self)

    re_self = check_spell(re_self)
    
    re_self = kkma_tokenizer(re_self)
    
    re_self = int_encode(re_self)
    
    intent = {0:'운행시작', 1:'가게전화',2:'가게도착',3:'픽업완료',4:'영수증번호',5:'소요시간선택', 6:'배달완료', 99:'fallback_intent'}

    if self == []:
        result = 99
    else:
        pad_self = padding(re_self)
        result = pred(pad_self)

    if result ==4:
        strip = re.sub(' +', '', self)
        numbers = re.findall("\d+",strip)[0]
        print(strip)
        print(numbers)
        if len(numbers) != 4:
            numbers = [99]
    else:
        numbers = 0

    return result,intent[result],self,numbers

In [ ]:
m =  re.search("\d\d\d\d",strip)

In [108]:
%%time
text = "운행시작할게"
print(start_predict(text))

(0, '운행시작', '운행시작할게', 0)
CPU times: user 112 ms, sys: 969 µs, total: 113 ms
Wall time: 192 ms


In [141]:
%%time
text = "주문번호는300 3이야"
print(start_predict(text))

주문번호는3003이야
3003
(4, '영수증번호', '주문번호는300 3이야', '3003')
CPU times: user 48.2 ms, sys: 5.25 ms, total: 53.5 ms
Wall time: 187 ms
